In [ ]:
%cd /content/drive/MyDrive/COMP 550 Project/data/

/content/drive/MyDrive/COMP 550 Project/data


In [ ]:
import pandas as pd

main_data = pd.read_csv('./isarcasmeval_train.csv')
main_data.dropna(subset=['tweet'], inplace=True)
main_data.head()

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
!pip install emoji contractions

In [ ]:
# replace user tags with @user, consecutive whitespaces to single space,
# surround punctuations with space as in https://aclanthology.org/2022.semeval-1.133.pdf
# also replace emojis with text desc using 'emoji'
# !pip install emoji contractions

import re
import emoji
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

def preprocess_tweet(tweet):
  # Replace tagged users with @user
  tweet = re.sub(r'@[\w]+', '@user', tweet)

  # Replace multiple white spaces with a single space
  tweet = re.sub(r'\s+', ' ', tweet)

  # Remove more than two successive occurrences of any punctuation
  tweet = re.sub(r'(\s*([.,!?;:()<>{}[\]\-+=~_$%^&*]\s*){2,})', r'\1', tweet)

  # Replace emoji with emoji descriptor
  tweet = emoji.demojize(tweet)

  # Put a single white space around punctuations
  tweet = re.sub(r'([.,!?;:()<>{}[\]\-+=~_$%^&*])', r' \1 ', tweet)

  # expand contractions
  tweet = expand_contractions(tweet)

  return tweet

# main_data['process_tweet'] = [preprocess_tweet(x) for x in main_data['tweet']]

In [ ]:
import json

data2 = []
with open('./sarcasmheadlines_train.json') as f:
  for line in f:
    data2.append(json.loads(line))
data2 = pd.DataFrame(data2)
data2['text'] = data2['headline']
data2['sarcasm_label'] = data2['is_sarcastic']
data2.drop(columns=['headline', 'is_sarcastic', 'article_link'], inplace=True)
data2.head()

,text,sarcasm_label
0,thirtysomething scientists unveil doomsday clo...,1
1,dem rep. totally nails why congress is falling...,0
2,eat your veggies: 9 deliciously different recipes,0
3,inclement weather prevents liar from getting t...,1
4,mother comes pretty close to using word 'strea...,1


In [ ]:
data3 = pd.read_csv('./semeval2018_train.txt', sep='\t')
data3['text'] = data3['Tweet text']
data3['sarcasm_label'] = data3['Label']
data3.drop(columns=['Tweet text', 'Label', 'Tweet index'], inplace=True)
data3.head()

,text,sarcasm_label
0,Sweet United Nations video. Just in time for C...,1
1,@mrdahl87 We are rumored to have talked to Erv...,1
2,Hey there! Nice to see you Minnesota/ND Winter...,1
3,3 episodes left I'm dying over here,0
4,I can't breathe! was chosen as the most notabl...,1


In [ ]:
data4 = pd.read_csv('./isarcasm_train.csv')
data4.head()

,text,sarcasm_label
0,sorry but sue from the mediweight advert looks...,0
1,@davesnyder since this is fantasy can it be an...,0
2,"Good times, fun times here in Grand old Britan...",1
3,I send light and love to all impacted by the #...,0
4,Another year closer to death,0


In [ ]:
data = main_data[['tweet', 'sarcastic']]
data.rename(columns={'tweet': 'text', 'sarcastic':'sarcasm_label'}, inplace=True)
data.head()

<ipython-input-9-35bd6030a05a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'tweet': 'text', 'sarcastic':'sarcasm_label'}, inplace=True)


,text,sarcasm_label
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1


In [ ]:
data_list = [data, data2, data3, data4]

## Train DL models

In [ ]:
all_data = pd.concat(data_list)
all_data['sarcasm_label'].value_counts()

0    21834
1    16883
Name: sarcasm_label, dtype: int64

In [ ]:
!pip install transformers[torch] accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import Trainer

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

## Test Dataset
class SarcasmTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred)

    return {"accuracy": accuracy,"f1_score":f1}

def labels(x):
    if x == 0:
        return 0
    else:
        return 1

## BERT-base

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', num_labels=2,)
                                        # loss_function_params={"weight": [0.75, 0.25]})

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

train = pd.concat([data, data2, data3, data4])
train['text'] = [preprocess_tweet(x) for x in train['text']]

train_tweets = train['text'].values.tolist()
train_labels = train['sarcasm_label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets,
                                                                      train_labels,
                                                                      test_size=0.2,
                                                                      random_state=42,
                                                                      stratify=train_labels)
train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.389200,0.325687,0.854726,0.826683
1000,0.309800,0.350167,0.877066,0.849842
1500,0.187000,0.287926,0.880553,0.857495
2000,0.172500,0.394854,0.887009,0.865818
2500,0.089500,0.440906,0.882877,0.862346


{'eval_loss': 0.28792619705200195,
 'eval_accuracy': 0.8805526859504132,
 'eval_f1_score': 0.8574949930673239,
 'eval_runtime': 51.8639,
 'eval_samples_per_second': 149.314,
 'eval_steps_per_second': 4.666,
 'epoch': 3.0}

In [ ]:
trainer.save_model('./bert-base-task-a/')

In [ ]:
model.save_pretrained('./bert-base-task-a/save_pretrained/')

## BERTTweet-base

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, max_seq_length=128)
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)

names = [name for name, param in model.named_parameters()]
trainable_params = names[-20:]

for name, param in model.named_parameters():
    if not name in trainable_params:
        param.requires_grad = False


train = pd.concat([data, data2, data3, data4])
train['text'] = [preprocess_tweet(x) for x in train['text']]

train_tweets = train['text'].values.tolist()
train_labels = train['sarcasm_label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels,
                                                                    test_size=0.2,random_state=42,stratify=train_labels)

train_encodings = tokenizer(train_tweets, padding='max_length', truncation=True, max_length=128)
val_encodings = tokenizer(val_tweets, padding='max_length', truncation=True, max_length=128)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir="berttweet-base-output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
    learning_rate=1e-4,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    gradient_accumulation_steps=16,                  # Number of steps to accumulate gradients before performing an optimizer step
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.478500,0.336912,0.855372,0.822729


{'eval_loss': 0.3369121253490448,
 'eval_accuracy': 0.8553719008264463,
 'eval_f1_score': 0.8227287116176005,
 'eval_runtime': 59.7584,
 'eval_samples_per_second': 129.589,
 'eval_steps_per_second': 16.199,
 'epoch': 3.0}

In [ ]:
trainer.save_model('./berttweet-base-task-a/')
model.save_pretrained('./berttweet-base-task-a/save_pretrained/')

## BERTweet-large

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large", use_fast=False, max_seq_length=512)
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-large", num_labels=2)

names = [name for name, param in model.named_parameters()]
trainable_params = names[-20:]

for name, param in model.named_parameters():
    if not name in trainable_params:
        param.requires_grad = False


train = pd.concat([data, data2, data3, data4])
train['text'] = [preprocess_tweet(x) for x in train['text']]

train_tweets = train['text'].values.tolist()
train_labels = train['sarcasm_label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels,
                                                                    test_size=0.2,random_state=42,stratify=train_labels)

train_encodings = tokenizer(train_tweets, padding='max_length', truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding='max_length', truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir="berttweet-large-output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
    learning_rate=1e-4,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    gradient_accumulation_steps=16,                  # Number of steps to accumulate gradients before performing an optimizer step
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

In [ ]:
trainer.save_model('./berttweet-large-task-a/')
model.save_pretrained('./berttweet-large-task-a/save_pretrained/')

## RoBERTa-base

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

train = pd.concat([data, data2, data3, data4])
train['text'] = [preprocess_tweet(x) for x in train['text']]

train_tweets = train['text'].values.tolist()
train_labels = train['sarcasm_label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels,
                                                                    test_size=0.2,random_state=42,stratify=train_labels)
train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)

training_args = TrainingArguments(
                      output_dir='./roberta-base-task-a/',
                      evaluation_strategy="steps",
                      num_train_epochs=3,
                      per_device_train_batch_size=32,
                      per_device_eval_batch_size=64,
                      warmup_steps=500, weight_decay=0.01,
                      logging_dir='./logs4', load_best_model_at_end=True)

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset,
                  compute_metrics=compute_metrics)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.508700,0.443664,0.836389,0.779076
1000,0.315600,0.284279,0.892691,0.867316
1500,0.210400,0.370699,0.887138,0.856815
2000,0.184600,0.354763,0.901343,0.880251
2500,0.093300,0.389481,0.899019,0.876850


{'eval_loss': 0.28427854180336,
 'eval_accuracy': 0.8926911157024794,
 'eval_f1_score': 0.8673159827558679,
 'eval_runtime': 62.6891,
 'eval_samples_per_second': 123.53,
 'eval_steps_per_second': 1.93,
 'epoch': 3.0}

In [ ]:
trainer.save_model('./roberta-base-task-a/')
model.save_pretrained('./roberta-base-task-a/save_pretrained/')

## XLNet

In [ ]:
!pip install sentencepiece

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from transformers import XLNetTokenizer
from transformers import XLNetForSequenceClassification

model_name = 'detecting-sarcasm'
task='sentiment'
MODEL = 'xlnet-base-cased'

tokenizer = XLNetTokenizer.from_pretrained(MODEL, num_labels=2)

model = XLNetForSequenceClassification.from_pretrained(MODEL)

train = pd.concat([data, data2, data3, data4])
train['text'] = [preprocess_tweet(x) for x in train['text']]

train_tweets = train['text'].values.tolist()
train_labels = train['sarcasm_label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels,
                                                                    test_size=0.2,random_state=42,stratify=train_labels)
train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512,return_tensors = 'pt')
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512,return_tensors = 'pt')

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)

training_args = TrainingArguments(output_dir='./xl-net/',
                                  num_train_epochs=3,
                                  per_device_train_batch_size=16,
                                  warmup_steps=500,
                                  weight_decay=0.01,
                                  logging_dir='./logs4',
                                  evaluation_strategy="steps",
                                  per_device_eval_batch_size=16,
                                  load_best_model_at_end=True,)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()

trainer.evaluate()
# preds = trainer.predict(test_dataset)
# preds = np.argmax(preds.predictions[:, 0:2], axis=-1)
# print(classification_report(y_test, preds))
# print(f1_score(y_test, preds))

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.590400,0.471450,0.792872,0.704604
1000,0.435700,0.400762,0.829675,0.774183
1500,0.420800,0.360460,0.841296,0.814826
2000,0.373700,0.388393,0.850465,0.808848
2500,0.314900,0.433081,0.852402,0.819972
3000,0.299400,0.364747,0.860795,0.837602


Checkpoint destination directory ./xl-net/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


## DeBERTa-base

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

task='sentiment'
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)


train = pd.concat([data, data2, data3, data4])
train['text'] = [preprocess_tweet(x) for x in train['text']]

train_tweets = train['text'].values.tolist()
train_labels = train['sarcasm_label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels,
                                                                    test_size=0.2,random_state=42,stratify=train_labels)
train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)


training_args = TrainingArguments(
    output_dir='./res', evaluation_strategy="steps", num_train_epochs=3, per_device_train_batch_size=16,
    per_device_eval_batch_size=64, warmup_steps=500, weight_decay=0.01,logging_dir='./logs4',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.572100,0.425972,0.811596,0.739882
1000,0.404300,0.466712,0.828642,0.772111
1500,0.371100,0.362537,0.856405,0.813046
2000,0.326500,0.329056,0.873450,0.842697
2500,0.259100,0.466578,0.867510,0.834195
3000,0.246400,0.341070,0.881457,0.852554
3500,0.242500,0.344940,0.879003,0.852278


Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.572100,0.425972,0.811596,0.739882
1000,0.404300,0.466712,0.828642,0.772111
1500,0.371100,0.362537,0.856405,0.813046
2000,0.326500,0.329056,0.873450,0.842697
2500,0.259100,0.466578,0.867510,0.834195
3000,0.246400,0.341070,0.881457,0.852554
3500,0.242500,0.344940,0.879003,0.852278
4000,0.203500,0.504977,0.888559,0.864287
4500,0.154700,0.400989,0.880811,0.860973
5000,0.138200,0.457854,0.884814,0.860756


{'eval_loss': 0.3290555477142334,
 'eval_accuracy': 0.8734504132231405,
 'eval_f1_score': 0.8426966292134832,
 'eval_runtime': 54.2325,
 'eval_samples_per_second': 142.793,
 'eval_steps_per_second': 2.231,
 'epoch': 3.0}

## Explainability

Done using https://github.com/hila-chefer/Transformer-Explainability and the Captum package

In [ ]:
!git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
os.chdir(f'./Transformer-Explainability')

Cloning into 'Transformer-Explainability'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 386 (delta 3), reused 2 (delta 0), pack-reused 377
Receiving objects: 100% (386/386), 3.85 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (194/194), done.


In [ ]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import (
    visualization
)
import torch

In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import BertPreTrainedModel
from BERT_explainability.modules.layers_ours import *
from BERT_explainability.modules.BERT.BERT import BertModel
from torch.nn import CrossEntropyLoss, MSELoss

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = Dropout(config.hidden_dropout_prob)
        self.classifier = Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def relprop(self, cam=None, **kwargs):
        cam = self.classifier.relprop(cam, **kwargs)
        cam = self.dropout.relprop(cam, **kwargs)
        cam = self.bert.relprop(cam, **kwargs)
        # print("conservation: ", cam.sum())
        return cam

In [ ]:
# use bert-base model and tokenizer fine-tuned on our dataset
model = model
tokenizer = tokenizer
model.eval()
explanations = Generator(model)

In [ ]:
text_batch = val_tweets[0]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].cuda()
attention_mask = encoding['attention_mask'].cuda()

true_class = 0

expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
expl = (expl - expl.min()) / (expl.max() - expl.min())

output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

classifications = ["1", "2"]
class_name = classifications[classification]

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class, 1, tokens, 1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('size', 0.12280241400003433), ('on', 0.12573294341564178), ('the', 0.05040403828024864), ('the', 0.030341923236846924), ('toulouse', 0.04929311200976372), ('team', 0.04048938676714897), (',', 0.18840965628623962), ('that', 0.24687208235263824), ('pack', 0.08874382823705673), ('is', 0.3363882303237915), ('mons', 1.0), ('##tro', 0.29786866903305054), ('##us', 0.6187840104103088), ('.', 0.34725049138069153), ('i', 0.05266257002949715), ('can', 0.0427778922021389), ("'", 0.020743733271956444), ('t', 0.026094427332282066), ('see', 0.0), ('a', 0.008165671490132809), ('welsh', 0.03817109018564224), ('region', 0.00046567944809794426), ('ever', 0.010123890824615955), ('winning', 0.006363356951624155), ('this', 0.05046335980296135), (',', 0.21455958485603333), ('money', 0.07296325266361237), ('talks', 0.06711722910404205), ('as', 0.23539891839027405), ('they', 0.03478177636861801), ('say', 0.02806495688855648), ('.', 0.18602493405342102), ('[SEP]', 0.02023129165172577)]


In [ ]:
text_batch = val_tweets[51]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].cuda()
attention_mask = encoding['attention_mask'].cuda()

true_class = 0

expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
expl = (expl - expl.min()) / (expl.max() - expl.min())

output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
class_name = classifications[classification]

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('we', 0.31147047877311707), ('need', 0.0), ('a', 0.20068085193634033), ('big', 0.9575868844985962), ('win', 0.1131214052438736), ('against', 0.39424678683280945), ('charlton', 1.0), ('next', 0.8191582560539246), ('weekend', 0.02106725051999092), ('[SEP]', 0.01705043762922287)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,3 (0.28),0,1.00,[CLS] we need a big win against charlton next weekend [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,3 (0.28),0,1.00,[CLS] we need a big win against charlton next weekend [SEP]


In [ ]:
#1
#Scores = [('[CLS]', 0.0), ('so', 0.3725809156894684), ('the', 0.17398375272750854), ('scottish', 0.13383816182613373), ('government', 0.07083307951688766), ('want', 0.1194242313504219), ('people', 0.06945335865020752), ('to', 0.022305114194750786), ('get', 0.09672229737043381), ('their', 0.03423971310257912), ('boost', 0.0), ('##er', 0.012416996993124485), ('shots', 0.06714650243520737), ('so', 0.5115399956703186), ('badly', 0.9102041721343994), ('that', 0.37313026189804077), ('the', 0.19795337319374084), ('website', 0.125137597322464), ('doesn', 1.0), ("'", 0.23679029941558838), ('t', 0.7906231880187988), ('even', 0.6365886330604553), ('work', 0.10039672255516052), ('[SEP]', 0.022343693301081657)]

#1
#Scores = wow = 1, so = 0.67, good = 0.38

#0
#Scores = big = 0.95, charlton = 1.0